<a href="https://colab.research.google.com/github/Alecia113/NLP-Ex/blob/main/E8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercise


## E1. Please describe two alternative solutions in order to prevent the zero count issue in n-gram language models. Please do not list them up but describe how they work.

E1. 请描述两个备选的解决方案，以防止n-gram语言模型中的零计数问题。请不要把它们列出来，而是描述它们是如何工作的。
[两个解决n-gram 模型中零计数问题。然后描述他们的工作流程】

Your answer:


## E2. Neural Language Model

You are required to modify the below example code that can be working with beam search (k > 1)

 神经语言模型
你需要修改下面的示例代码，可以用波束搜索（k > 1）工作。

Now, let's see how to build a language model for generating natural language text by implement and training state-of-the-art Recurrent Neural Network. The objective of this model is to generate new text, given that some input text is present. Lets start building the architecture.

现在，让我们看看如何通过实现和训练最先进的循环神经网络来建立一个生成自然语言文本的语言模型。这个模型的目标是在有一些输入文本的情况下，生成新的文本。让我们开始建立这个架构。


In [1]:
import numpy as np 

from numpy import array
from numpy import argmax
from numpy import log

Lets use a popular nursery rhyme — “Cat and Her Kittens” as our corpus. A corpus is defined as the collection of text documents.

让我们用一首流行的童谣--《猫和她的小猫》作为我们的语料库。语料库被定义为文本文件的集合。

In [2]:
import re

# Pad sequences to the max length   填充序列至最大长度
def pad_sequences_pre(input_sequences, maxlen):
    output = []
    for inp in input_sequences:
        if len(inp)< maxlen:
            output.append([0]*(maxlen-len(inp)) + inp)
        else:
            output.append(inp[:maxlen])
    return output

# Prepare the data  准备数据
def dataset_preparation(data):
    corpus = data.lower().split("\n")
    normalized_text=[]
    for string in corpus:
        tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
        normalized_text.append(tokens)
    tokenized_sentences=[sentence.strip().split(" ") for sentence in normalized_text]

    word_list_dict ={}
    for sent in tokenized_sentences:
        for word in sent:
            if word != "":
                word_list_dict[word] = 1
    word_list = list(word_list_dict.keys())
    word_to_index = {word:word_list.index(word) for word in word_list}

    total_words = len(word_list)+1

    # create input sequences using list of tokens  使用标记列表创建输入序列
    input_sequences = []
    for line in tokenized_sentences:
        token_list = []
        for word in line:
            if word!="":
                token_list.append(word_to_index[word])
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)

    # pad sequences 补充序列 
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences_pre(input_sequences, maxlen=max_sequence_len))

    # create predictors and label  创建预测器和标签
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

    return predictors, np.array(label), max_sequence_len, total_words, word_list, word_to_index

data = '''The cat and her kittens
They put on their mittens
To eat a christmas pie
The poor little kittens
They lost their mittens
And then they began to cry.

O mother dear, we sadly fear
We cannot go to-day,
For we have lost our mittens
If it be so, ye shall not go
For ye are naughty kittens'''

predictors, label, max_sequence_len, total_words, word_list, word_to_index = dataset_preparation(data)

In [ ]:
# import torch
# torch.cuda.is_available()

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define the model
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim_1, hidden_dim_2, total_words):
        super(LSTMTagger, self).__init__()
        self.hidden_dim_1 = hidden_dim_1
        self.hidden_dim_2 = hidden_dim_2
        self.word_embeddings = nn.Embedding(total_words, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim_1, batch_first=True)  
        self.lstm2 = nn.LSTM(hidden_dim_1, hidden_dim_2, batch_first=True)  
        self.hidden2tag = nn.Linear(hidden_dim_2, total_words)


    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out_1, _ = self.lstm1(embeds)
        lstm_out_2, _ = self.lstm2(lstm_out_1)
        tag_space = self.hidden2tag(lstm_out_2[:,-1,:])
        # The reason we are using log_softmax here is that we want to calculate -log(p) and find the minimum score      
        #我们在这里使用log_softmax的原因是，我们想计算-log(p)，并找到最小分值。                                 
        tag_scores = F.log_softmax(tag_space, dim=1)      
        return tag_scores

# Parameter setting
EMBEDDING_DIM = 10
HIDDEN_DIM_1 = 150
HIDDEN_DIM_2 = 100
batch_size=predictors.shape[0]

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM_1, HIDDEN_DIM_2, total_words).cuda() #10,150,100,43 这东西colab没开GPU还用不了……
loss_function = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


sentence =torch.from_numpy(predictors).cuda().to(torch.int64)
targets = torch.from_numpy(label).cuda().to(torch.int64)


# Training
for epoch in range(100):  

    model.train()
    model.zero_grad()       
    tag_scores = model(sentence)
    loss = loss_function(tag_scores, targets)
    loss.backward()
    optimizer.step()


    if epoch % 10 == 9:
        model.eval()
        _, predicted = torch.max(tag_scores, 1)
        prediction = predicted.view(-1).cpu().numpy()
        t = targets.view(-1).cpu().numpy()
        acc = accuracy_score(prediction,t)
        print('Epoch: %d, training loss: %.4f, training acc: %.2f%%'%(epoch+1,loss.item(),100*acc))



Epoch: 10, training loss: 3.6545, training acc: 8.33%
Epoch: 20, training loss: 3.4209, training acc: 10.42%
Epoch: 30, training loss: 2.9872, training acc: 14.58%
Epoch: 40, training loss: 2.5351, training acc: 22.92%
Epoch: 50, training loss: 2.1439, training acc: 52.08%
Epoch: 60, training loss: 1.8357, training acc: 68.75%
Epoch: 70, training loss: 1.5770, training acc: 77.08%
Epoch: 80, training loss: 1.4555, training acc: 79.17%
Epoch: 90, training loss: 1.2344, training acc: 81.25%
Epoch: 100, training loss: 1.0418, training acc: 89.58%


The code below only works with k=1, it does not store the candidates. You need to modify the code to make it work with k > 1.

下面的代码只在k=1的情况下工作，它不存储候选人。你需要修改代码以使其在k>1的情况下工作。

In [ ]:
'''
假设k（beam size) = 2
然后取前k words 然后算分数。然后下个词的前k个词计算分数。
在这些𝒌方假设中，只保留得分最高的k；选择得分最高的假说 !
感觉是光波是几就取几个最大值（总共的最大值）
'''

In [4]:
# define a sequence of 10 words over a vocab of 5 words
from numpy import array
from numpy import log
data1 = [[0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1],
        [0.1, 0.2, 0.3, 0.4, 0.5],
        [0.5, 0.4, 0.3, 0.2, 0.1]]
data1 = array(data1)
for step,row in enumerate(data1):  #第几次，然后第几行 0-9次 每次的每行
  # print(step)
   print('\n')
  # # print(row)
  #  print(row[0])  # 一个小数
  #  print((-log(row[0])))  #每行的第几个 的log值
  

In [5]:
data1

array([[0.1, 0.2, 0.3, 0.4, 0.5],
       [0.5, 0.4, 0.3, 0.2, 0.1],
       [0.1, 0.2, 0.3, 0.4, 0.5],
       [0.5, 0.4, 0.3, 0.2, 0.1],
       [0.1, 0.2, 0.3, 0.4, 0.5],
       [0.5, 0.4, 0.3, 0.2, 0.1],
       [0.1, 0.2, 0.3, 0.4, 0.5],
       [0.5, 0.4, 0.3, 0.2, 0.1],
       [0.1, 0.2, 0.3, 0.4, 0.5],
       [0.5, 0.4, 0.3, 0.2, 0.1]])

In [ ]:
seed_text = "we naughty"
next_words = 3
max_sequence_len = max_sequence_len
k = 3

seed_candidates = [(seed_text, .0)] 
seed_candidates
for _ in range(next_words): #3  _ = 0,1,2


0
1
2


In [ ]:
predictors, label, max_sequence_len, total_words, word_list, word_to_index = dataset_preparation(data)
token_list = [word_to_index[word] for word in seed_text.split()]
token_list

[24, 41]

In [12]:
predicted[0]

array([-9.145516 , -8.109267 , -7.940792 , -6.1404257, -1.2899072,
       -3.022852 , -4.556643 , -2.445673 , -2.6552176, -5.9843636,
       -4.4980793, -7.4669857, -5.933133 , -3.6852694, -6.186511 ,
       -7.40804  , -5.855938 , -3.9005868, -7.244426 , -2.4567208,
       -8.26527  , -9.467429 , -6.2680283, -4.155842 , -3.0354843,
       -6.5130568, -8.289023 , -5.483369 , -1.6300032, -7.070237 ,
       -8.9612665, -5.156124 , -5.3352137, -8.622119 , -6.321787 ,
       -4.148204 , -3.5803058, -4.0651474, -7.588977 , -7.5636854,
       -5.81188  , -5.2577076, -8.663109 ], dtype=float32)

In [10]:
np.argsort(predicted[0])

array([21,  0, 30, 42, 33, 26, 20,  1,  2, 38, 39, 11, 15, 18, 29, 25, 34,
       22, 14,  3,  9, 12, 16, 40, 27, 32, 41, 31,  6, 10, 23, 35, 37, 17,
       13, 36, 24,  5,  8, 19,  7, 28,  4])

In [13]:
np.argsort(predicted[0])[-1:]

array([4])

In [15]:
np.argsort(predicted[0])[-3:]

array([ 7, 28,  4])

In [21]:
top_k = np.argsort(predicted[0])[-1:]


In [24]:
N= [(id, predicted[0][id]) for id in top_k]  #
N

[(4, -1.2899072)]

In [29]:
id, s = get_topK(predicted, k)[0]     #7  -2.445673
s

-2.445673

In [30]:
def get_topK(predicted, k=1):  #那这里需要传递一个k
    
    # Get the index of the highest k index # 获得最高的k指数的索引
    # Since the input is just one sentence, we can use [0] to extract the prediction result
    # # 由于输入的只是一个句子，我们可以用[0]来提取预测结果
    top_k = np.argsort(predicted[0])[-k:] #返回从小到大拍的位置; 倒着取就是最大的呗。 array

    # return a list of tuple  # 返回一个元组的列表
    # tuple[0]:word_id, tuple[1]:log(p)
    return [(id, predicted[0][id]) for id in top_k] # N = xxx  [(4, -1.2899072)] 合在了一起。把位置和最大的值取出来了。

id, s = get_topK(predicted, k)[0]   #所以三个的话，这里要有个for循环。

In [35]:
def get_topK(predicted, k=1):  #那这里需要传递一个k
    
    # Get the index of the highest k index # 获得最高的k指数的索引
    # Since the input is just one sentence, we can use [0] to extract the prediction result
    # # 由于输入的只是一个句子，我们可以用[0]来提取预测结果
    top_k = np.argsort(predicted[0])[-k:] #返回从小到大拍的位置; 倒着取就是最大的呗。 array

    # return a list of tuple  # 返回一个元组的列表
    # tuple[0]:word_id, tuple[1]:log(p)
    return [(id, predicted[0][id]) for id in top_k] # N = xxx  [(4, -1.2899072)] 合在了一起。把位置和最大的值取出来了。

id, s = get_topK(predicted, k)[0]   #所以三个的话，这里要有个for循环。

In [49]:
top_k = np.argsort(predicted[0])[-k:]  
top_k

array([ 7, 28,  4])

In [50]:
predicted

array([[-9.145516 , -8.109267 , -7.940792 , -6.1404257, -1.2899072,
        -3.022852 , -4.556643 , -2.445673 , -2.6552176, -5.9843636,
        -4.4980793, -7.4669857, -5.933133 , -3.6852694, -6.186511 ,
        -7.40804  , -5.855938 , -3.9005868, -7.244426 , -2.4567208,
        -8.26527  , -9.467429 , -6.2680283, -4.155842 , -3.0354843,
        -6.5130568, -8.289023 , -5.483369 , -1.6300032, -7.070237 ,
        -8.9612665, -5.156124 , -5.3352137, -8.622119 , -6.321787 ,
        -4.148204 , -3.5803058, -4.0651474, -7.588977 , -7.5636854,
        -5.81188  , -5.2577076, -8.663109 ]], dtype=float32)

In [56]:
word_to_index.items()


dict_items([('the', 0), ('cat', 1), ('and', 2), ('her', 3), ('kittens', 4), ('they', 5), ('put', 6), ('on', 7), ('their', 8), ('mittens', 9), ('to', 10), ('eat', 11), ('a', 12), ('christmas', 13), ('pie', 14), ('poor', 15), ('little', 16), ('lost', 17), ('then', 18), ('began', 19), ('cry', 20), ('o', 21), ('mother', 22), ('dear', 23), ('we', 24), ('sadly', 25), ('fear', 26), ('cannot', 27), ('go', 28), ('day', 29), ('for', 30), ('have', 31), ('our', 32), ('if', 33), ('it', 34), ('be', 35), ('so', 36), ('ye', 37), ('shall', 38), ('not', 39), ('are', 40), ('naughty', 41)])

In [57]:
top_k = np.argsort(predicted[0])[-k:]   #array([32, 10, 29]) 计算数组排序的下标;  目前k=3 array([ 7, 28,  4]))排序后的最大三个位置
predicted 
#ind_to_word
predicted_ind = id    #7  这里最少是k个id啊；应该是个list
for word, index in word_to_index.items(): #word_to_index.items()变成一对一对的了
    if index == predicted_ind:    #然后把这个单词和index找出来，就是把index投进去
      print(word)
      print(index)


on
7


In [64]:
score

4.198000431060791

In [63]:
successives.append((seed_text + ' ' + output_word, score - s))  
successives

[('we naughty on their cry', 6.302371978759766),
 ('we naughty on their cry', 6.302371978759766),
 ('we naughty on their cry', 6.302371978759766)]

In [66]:
ordered = sorted(successives, key=lambda tup: tup[1])
ordered

[('we naughty on their cry', 6.302371978759766),
 ('we naughty on their cry', 6.302371978759766),
 ('we naughty on their cry', 6.302371978759766)]

In [71]:
seed_candidates = ordered[:3]  #k = 1就是一个
seed_candidates


[('we naughty on their cry', 6.302371978759766),
 ('we naughty on their cry', 6.302371978759766),
 ('we naughty on their cry', 6.302371978759766)]

In [72]:
print(seed_candidates[0][0])  #只打出来值。不要后面的数

we naughty on their cry


In [61]:
seed_text = "we naughty"
next_words = 3
max_sequence_len = max_sequence_len
#k = 3 这就恒定了
#目前是初始化，[('we naughty', 0.0)]
seed_candidates = [(seed_text, .0)] #"we naughty"  [('we naughty they our kittens', 8.618810653686523)]把里面的数据取出来
for _ in range(next_words): #3 #3  _ = 0,1,2
    successives = [] #逐次导数？
    # if k = 1, len(seed_candidates) will always be 1 # 如果k = 1，len(seed_candidates)将总是1
    for i in range(len(seed_candidates)): #len(sequences) 目前里面一句话1 这里要改。
        seed_text, score = seed_candidates[i]  #之前是文本和分数。初级然后慢慢迭代更新。
     
        token_list = [word_to_index[word] for word in seed_text.split()]  #[24, 41] 目前只有两个词。所以分开找we': 24,'naughty': 41,， 变成index
        token_list = pad_sequences_pre([token_list], maxlen=max_sequence_len-1)   #7扩充到最大长度 list

        seed_input = torch.from_numpy(np.array(token_list)).cuda().to(torch.int64)  #然后变成tensor 长度还是7 里面还是那两个单词。排在倒数的位置
        predicted = model(seed_input).cpu().detach().numpy()  #跑了那个模型RNN变array了 变成（1，43） 这个向量维度是43，现有句子。

        # Since it it only works with k = 1, we can simply use [0] to get the word id and log(p)
          # 因为它只对k = 1起作用，我们可以简单地用[0]来获得单词id和log(p)
        # However, if k = 3, you can't simply use [0] to get the candidates
          # 然而，如果k = 3，就不能简单地用[0]来获得候选者了
        id, s = get_topK(predicted, k)[0]  #把这句话的array都丢进去做gettopK了
        # get the output word #获得输出字数
        output_word = ind_to_word(id)
        # put the word into the sentence input#把这个词放到句子的输入中
        # calcualte the accumulated score by -log(p)#用-log(p)计算累计得分。 每行的第几个 的log值 
        successives.append((seed_text + ' ' + output_word, score - s))  #candidate = [seq + [j], score + (-log(row[j])) ]、、all_candidates.append(candidate)
  #这后面一样 #每行的第几个 的log值  #score最开始的得分
    # Get the lowest k accumulated scores (highest k accumulated probabilities)
    ## 获得最低的k个累积分数（最高的k个累积概率）。
    # Then, make them as the seed_candidate for the next word to predict
    # 然后，把它们作为下一个要预测的词的种子_候选者
    ordered = sorted(successives, key=lambda tup: tup[1])
    seed_candidates = ordered[:k]
print(seed_candidates[0][0])


we naughty on their cry


In [ ]:
top_k = np.argsort(predicted[0])[-k:]   #array([32, 10, 29]) 计算数组排序的下标;
predicted 

In [ ]:
predicted_ind = id
for word, index in word_to_index.items():
    if index == predicted_ind:
      print(word)


In [60]:
# convert index to word  将索引转换为单词 索引等于预测的就返回单词
def ind_to_word(predicted_ind):
    for word, index in word_to_index.items():
        if index == predicted_ind:
            return word
    return ""    


# get the top k most predicted results 获得前k个最有预测性的结果
def get_topK(predicted, k=1):  #那这里需要传递一个k
    
    # Get the index of the highest k index # 获得最高的k指数的索引
    # Since the input is just one sentence, we can use [0] to extract the prediction result
    # # 由于输入的只是一个句子，我们可以用[0]来提取预测结果
    top_k = np.argsort(predicted[0])[-k:]

    # return a list of tuple  # 返回一个元组的列表
    # tuple[0]:word_id, tuple[1]:log(p)
    return [(id, predicted[0][id]) for id in top_k]

# To-Do: modify this function# 待办事项：修改此功能
# Generate text, currently only works with k=1 # 生成文本，目前只在k=1时有效 
# Hint: The easist way is modifying the code from line 40-47, but it is not compulsory
## 提示：最简单的方法是修改第40-47行的代码，但这并不是强制性的
# beam search 光束搜索

In [ ]:
# convert index to word  将索引转换为单词 索引等于预测的就返回单词
def ind_to_word(predicted_ind):
    for word, index in word_to_index.items():
        if index == predicted_ind:
            return word
    return ""    


# get the top k most predicted results 获得前k个最有预测性的结果
def get_topK(predicted, k=1):  #那这里需要传递一个k
    
    # Get the index of the highest k index # 获得最高的k指数的索引
    # Since the input is just one sentence, we can use [0] to extract the prediction result
    # # 由于输入的只是一个句子，我们可以用[0]来提取预测结果
    top_k = np.argsort(predicted[0])[-k:]

    # return a list of tuple  # 返回一个元组的列表
    # tuple[0]:word_id, tuple[1]:log(p)
    return [(id, predicted[0][id]) for id in top_k]

# To-Do: modify this function# 待办事项：修改此功能
# Generate text, currently only works with k=1 # 生成文本，目前只在k=1时有效 
# Hint: The easist way is modifying the code from line 40-47, but it is not compulsory
## 提示：最简单的方法是修改第40-47行的代码，但这并不是强制性的
# beam search 光束搜索

def generate_text(seed_text, next_words, max_sequence_len, k):

    seed_candidates = [(seed_text, .0)]
    for _ in range(next_words):
        successives = []
        # if k = 1, len(seed_candidates) will always be 1 # 如果k = 1，len(seed_candidates)将总是1
        for i in range(len(seed_candidates)): #len(sequences)
            seed_text, score = seed_candidates[i]
            for j in range(len(row)):
              candidate = [seq + [j], score + (-log(row[j])) ]  #we are summing up the negative log, so we need to find the minimum score(which is the highest prob)
              successives.append(candidate)
 
            token_list = [word_to_index[word] for word in seed_text.split()]
            token_list = pad_sequences_pre([token_list], maxlen=max_sequence_len-1)

            seed_input = torch.from_numpy(np.array(token_list)).cuda().to(torch.int64)
            predicted = model(seed_input).cpu().detach().numpy()


            # Since it it only works with k = 1, we can simply use [0] to get the word id and log(p)
             # 因为它只对k = 1起作用，我们可以简单地用[0]来获得单词id和log(p)
            # However, if k = 3, you can't simply use [0] to get the candidates
             # 然而，如果k = 3，就不能简单地用[0]来获得候选者了
            id, s = get_topK(predicted, k)[0]
            # get the output word #获得输出字数
            output_word = ind_to_word(id)
            # put the word into the sentence input#把这个词放到句子的输入中
            # calcualte the accumulated score by -log(p)#用-log(p)计算累计得分。
            successives.append((seed_text + ' ' + output_word, score - s)) 
      #这后面一样
        # Get the lowest k accumulated scores (highest k accumulated probabilities)
        ## 获得最低的k个累积分数（最高的k个累积概率）。
        # Then, make them as the seed_candidate for the next word to predict
        # 然后，把它们作为下一个要预测的词的种子_候选者
        ordered = sorted(successives, key=lambda tup: tup[1])
        seed_candidates = ordered[:k]

    return seed_candidates[0][0]


print(generate_text("we naughty", 3, max_sequence_len, 1))
print(generate_text("we naughty", 3, max_sequence_len, 3))

# Please note that it can happen that k=1 and k=3 have the same output because this is only a small dataset.
#在解码器的每一步，跟踪k个最有可能的部分序列（我们称之为假设）--K是波束大小（在实践中约为5至10）
#选出前k个词并计算分数

In [ ]:
# convert index to word  将索引转换为单词 索引等于预测的就返回单词
def ind_to_word(predicted_ind):
    for word, index in word_to_index.items():
        if index == predicted_ind:
            return word
    return ""    


# get the top k most predicted results 获得前k个最有预测性的结果
def get_topK(predicted, k=1):
    
    # Get the index of the highest k index # 获得最高的k指数的索引
    # Since the input is just one sentence, we can use [0] to extract the prediction result
    # # 由于输入的只是一个句子，我们可以用[0]来提取预测结果
    top_k = np.argsort(predicted[0])[-k:]

    # return a list of tuple  # 返回一个元组的列表
    # tuple[0]:word_id, tuple[1]:log(p)
    return [(id, predicted[0][id]) for id in top_k]


# To-Do: modify this function# 待办事项：修改此功能
# Generate text, currently only works with k=1 # 生成文本，目前只在k=1时有效 
# Hint: The easist way is modifying the code from line 40-47, but it is not compulsory
## 提示：最简单的方法是修改第40-47行的代码，但这并不是强制性的

def generate_text(seed_text, next_words, max_sequence_len, k=1):

    seed_candidates = [(seed_text, .0)]
    for _ in range(next_words):
        successives = []
        # if k = 1, len(seed_candidates) will always be 1 # 如果k = 1，len(seed_candidates)将总是1
        for i in range(len(seed_candidates)):
            seed_text, score = seed_candidates[i]
            token_list = [word_to_index[word] for word in seed_text.split()]
            token_list = pad_sequences_pre([token_list], maxlen=max_sequence_len-1)

            seed_input = torch.from_numpy(np.array(token_list)).cuda().to(torch.int64)
            predicted = model(seed_input).cpu().detach().numpy()


            # Since it it only works with k = 1, we can simply use [0] to get the word id and log(p)
             # 因为它只对k = 1起作用，我们可以简单地用[0]来获得单词id和log(p)
            # However, if k = 3, you can't simply use [0] to get the candidates
             # 然而，如果k = 3，就不能简单地用[0]来获得候选者了
            id, s = get_topK(predicted, k)[0]
            # get the output word #获得输出字数
            output_word = ind_to_word(id)
            # put the word into the sentence input#把这个词放到句子的输入中
            # calcualte the accumulated score by -log(p)#用-log(p)计算累计得分。
            successives.append((seed_text + ' ' + output_word, score - s)) 

        # Get the lowest k accumulated scores (highest k accumulated probabilities)
        ## 获得最低的k个累积分数（最高的k个累积概率）。
        # Then, make them as the seed_candidate for the next word to predict
        # 然后，把它们作为下一个要预测的词的种子_候选者
        ordered = sorted(successives, key=lambda tup: tup[1])
        seed_candidates = ordered[:k]

    return seed_candidates[0][0]


print(generate_text("we naughty", 3, max_sequence_len, k=1))
print(generate_text("we naughty", 3, max_sequence_len, k=3))

# Please note that it can happen that k=1 and k=3 have the same output because this is only a small dataset.


we naughty go to to
we naughty her kittens go


**Sample Output** (Your output would be different, it is based on the trained model)


```
we naughty lost their mittens
```

